# Librerias a utilizas

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt

# Descripcion del data set

El problema consiste en predecir la probabilidad de atraso de los vuelos que aterrizan o despegan del aeropuerto de Santiago de
Chile (SCL). Para eso les entregamos un dataset usando datos públicos y reales donde cada fila corresponde a un vuelo que
aterrizó o despegó de SCL. Para cada vuelo se cuenta con la siguiente información:

- **Fecha-I** : Fecha y hora programada del vuelo.
- **Vlo-I** : Número de vuelo programado.
- **Ori-I** : Código de ciudad de origen programado.
- **Des-I** : Código de ciudad de destino programado.
- **Emp-I** : Código aerolínea de vuelo programado.
- **Fecha-O** : Fecha y hora de operación del vuelo.
- **Vlo-O** : Número de vuelo de operación del vuelo.
- **Ori-O** : Código de ciudad de origen de operación
- **Des-O** : Código de ciudad de destino de operación.
- **Emp-O** : Código aerolínea de vuelo operado.
- **DIA** : Día del mes de operación del vuelo.
- **MES** : Número de mes de operación del vuelo.
- **AÑO** : Año de operación del vuelo.
- **DIANOM** : Día de la semana de operación del vuelo.
- **TIPOVUELO** : Tipo de vuelo, I =Internacional, N =Nacional.
- **OPERA** : Nombre de aerolínea que opera.
- **SIGLAORI** : Nombre ciudad origen.
- **SIGLADES** : Nombre ciudad destino.

**Desafio**

1. ¿Cómo se distribuyen los datos? ¿Qué te llama la atención o cuál es tu conclusión sobre esto?
2. Genera las columnas adicionales y luego expórtelas en un archivo **synthetic_features.csv** :
  - **temporada_alta** : 1 si Fecha-I está entre 15-Dic y 3-Mar, o 15-Jul y 31-Jul, o 11-Sep y 30-Sep, 0 si no.
  - **dif_min** : diferencia en minutos entre Fecha-O y Fecha-I .
  - **atraso_15** : 1 si dif_min > 15, 0 si no.
  - **periodo_dia** : mañana (entre 5:00 y 11:59), tarde (entre 12:00 y 18:59) y noche (entre 19:00 y 4:59), en base a Fecha-I 
  

3. ¿Cómo se compone la tasa de atraso por destino, aerolínea, mes del año, día de la semana, temporada, tipo de vuelo?
¿Qué variables esperarías que más influyeran en predecir atrasos?
4. Entrena uno o varios modelos (usando el/los algoritmo(s) que prefieras) para estimar la probabilidad de atraso de un vuelo.
Siéntete libre de generar variables adicionales y/o complementar con variables externas.
5. Evalúa tu modelo. ¿Qué performance tiene? ¿Qué métricas usaste para evaluar esa performance y por qué? ¿Por qué
elegiste ese algoritmo en particular? ¿Qué variables son las que más influyen en la predicción? ¿Cómo podrías mejorar la
performance?




# Exploramos el dataset

- ¿Cómo se distribuyen los datos? ¿Qué te llama la atención o cuál es tu conclusión sobre esto?

In [2]:
df = pd.read_csv('dataset_SCL.csv')
df.head()

/home/juliom86/.pyenv/versions/3.8.6/envs/lewagon/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (1,6) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Fecha-I,Vlo-I,Ori-I,Des-I,Emp-I,Fecha-O,Vlo-O,Ori-O,Des-O,Emp-O,DIA,MES,AÑO,DIANOM,TIPOVUELO,OPERA,SIGLAORI,SIGLADES
0,2017-01-01 23:30:00,226,SCEL,KMIA,AAL,2017-01-01 23:33:00,226,SCEL,KMIA,AAL,1,1,2017,Domingo,I,American Airlines,Santiago,Miami
1,2017-01-02 23:30:00,226,SCEL,KMIA,AAL,2017-01-02 23:39:00,226,SCEL,KMIA,AAL,2,1,2017,Lunes,I,American Airlines,Santiago,Miami
2,2017-01-03 23:30:00,226,SCEL,KMIA,AAL,2017-01-03 23:39:00,226,SCEL,KMIA,AAL,3,1,2017,Martes,I,American Airlines,Santiago,Miami
3,2017-01-04 23:30:00,226,SCEL,KMIA,AAL,2017-01-04 23:33:00,226,SCEL,KMIA,AAL,4,1,2017,Miercoles,I,American Airlines,Santiago,Miami
4,2017-01-05 23:30:00,226,SCEL,KMIA,AAL,2017-01-05 23:28:00,226,SCEL,KMIA,AAL,5,1,2017,Jueves,I,American Airlines,Santiago,Miami


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68206 entries, 0 to 68205
Data columns (total 18 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Fecha-I    68206 non-null  object
 1   Vlo-I      68206 non-null  object
 2   Ori-I      68206 non-null  object
 3   Des-I      68206 non-null  object
 4   Emp-I      68206 non-null  object
 5   Fecha-O    68206 non-null  object
 6   Vlo-O      68205 non-null  object
 7   Ori-O      68206 non-null  object
 8   Des-O      68206 non-null  object
 9   Emp-O      68206 non-null  object
 10  DIA        68206 non-null  int64 
 11  MES        68206 non-null  int64 
 12  AÑO        68206 non-null  int64 
 13  DIANOM     68206 non-null  object
 14  TIPOVUELO  68206 non-null  object
 15  OPERA      68206 non-null  object
 16  SIGLAORI   68206 non-null  object
 17  SIGLADES   68206 non-null  object
dtypes: int64(3), object(15)
memory usage: 9.4+ MB


In [4]:
df.shape

(68206, 18)

In [5]:
df.dtypes.value_counts()

object    15
int64      3
dtype: int64

In [6]:
df.isnull().sum().sort_values(ascending=False)/len(df)

Vlo-O        0.000015
Fecha-I      0.000000
DIA          0.000000
SIGLAORI     0.000000
OPERA        0.000000
TIPOVUELO    0.000000
DIANOM       0.000000
AÑO          0.000000
MES          0.000000
Emp-O        0.000000
Vlo-I        0.000000
Des-O        0.000000
Ori-O        0.000000
Fecha-O      0.000000
Emp-I        0.000000
Des-I        0.000000
Ori-I        0.000000
SIGLADES     0.000000
dtype: float64

- La respuesta  la primer pregunta es que tenemos un total de 15 variables objetos, entre ellas categoricas que tendriamos que trabajarlas a cada una.
- La fecha se tiene que trabajar en formato datetime, tendremos que ver la diferencia entre dia nominal y dia
- Tenemos muy pocas variables nulas, asi que las podemos eliminar tranquilamente

In [7]:
df['Fecha-I'] = pd.to_datetime(df['Fecha-I'], format = '%Y-%m-%d %H:%M:%S')
df['Fecha-O'] = pd.to_datetime(df['Fecha-O'], format = '%Y-%m-%d %H:%M:%S')

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68206 entries, 0 to 68205
Data columns (total 18 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Fecha-I    68206 non-null  datetime64[ns]
 1   Vlo-I      68206 non-null  object        
 2   Ori-I      68206 non-null  object        
 3   Des-I      68206 non-null  object        
 4   Emp-I      68206 non-null  object        
 5   Fecha-O    68206 non-null  datetime64[ns]
 6   Vlo-O      68205 non-null  object        
 7   Ori-O      68206 non-null  object        
 8   Des-O      68206 non-null  object        
 9   Emp-O      68206 non-null  object        
 10  DIA        68206 non-null  int64         
 11  MES        68206 non-null  int64         
 12  AÑO        68206 non-null  int64         
 13  DIANOM     68206 non-null  object        
 14  TIPOVUELO  68206 non-null  object        
 15  OPERA      68206 non-null  object        
 16  SIGLAORI   68206 non-null  object       

In [9]:
df['dif_min'] = (df['Fecha-O'] - df['Fecha-I'])/ np.timedelta64(1, 'h') * 60

In [10]:
df.head()

,Fecha-I,Vlo-I,Ori-I,Des-I,Emp-I,Fecha-O,Vlo-O,Ori-O,Des-O,Emp-O,DIA,MES,AÑO,DIANOM,TIPOVUELO,OPERA,SIGLAORI,SIGLADES,dif_min
0,2017-01-01 23:30:00,226,SCEL,KMIA,AAL,2017-01-01 23:33:00,226,SCEL,KMIA,AAL,1,1,2017,Domingo,I,American Airlines,Santiago,Miami,3.0
1,2017-01-02 23:30:00,226,SCEL,KMIA,AAL,2017-01-02 23:39:00,226,SCEL,KMIA,AAL,2,1,2017,Lunes,I,American Airlines,Santiago,Miami,9.0
2,2017-01-03 23:30:00,226,SCEL,KMIA,AAL,2017-01-03 23:39:00,226,SCEL,KMIA,AAL,3,1,2017,Martes,I,American Airlines,Santiago,Miami,9.0
3,2017-01-04 23:30:00,226,SCEL,KMIA,AAL,2017-01-04 23:33:00,226,SCEL,KMIA,AAL,4,1,2017,Miercoles,I,American Airlines,Santiago,Miami,3.0
4,2017-01-05 23:30:00,226,SCEL,KMIA,AAL,2017-01-05 23:28:00,226,SCEL,KMIA,AAL,5,1,2017,Jueves,I,American Airlines,Santiago,Miami,-2.0


In [11]:
# temporada_alta : 1 si Fecha-I está entre 15-Dic y 3-Mar, o 15-Jul y 31-Jul, o 11-Sep y 30-Sep, 0 si no.
#ya sacamos las diferencias de minutos, falraria ver los negativos#dif_min : diferencia en minutos entre Fecha-O y Fecha-I .
#atraso_15 : 1 si dif_min > 15, 0 si no.
#periodo_dia : mañana (entre 5:00 y 11:59), tarde (entre 12:00 y 18:59) y noche (entre 19:00 y 4:59), en base a Fecha-I

In [12]:
# funcion para retraso
def retraso(x):
    if x > 15:
        return 1
    return 0

In [13]:
df['atraso_15'] = df['dif_min'].apply(retraso)

In [14]:
df.head()

,Fecha-I,Vlo-I,Ori-I,Des-I,Emp-I,Fecha-O,Vlo-O,Ori-O,Des-O,Emp-O,DIA,MES,AÑO,DIANOM,TIPOVUELO,OPERA,SIGLAORI,SIGLADES,dif_min,atraso_15
0,2017-01-01 23:30:00,226,SCEL,KMIA,AAL,2017-01-01 23:33:00,226,SCEL,KMIA,AAL,1,1,2017,Domingo,I,American Airlines,Santiago,Miami,3.0,0
1,2017-01-02 23:30:00,226,SCEL,KMIA,AAL,2017-01-02 23:39:00,226,SCEL,KMIA,AAL,2,1,2017,Lunes,I,American Airlines,Santiago,Miami,9.0,0
2,2017-01-03 23:30:00,226,SCEL,KMIA,AAL,2017-01-03 23:39:00,226,SCEL,KMIA,AAL,3,1,2017,Martes,I,American Airlines,Santiago,Miami,9.0,0
3,2017-01-04 23:30:00,226,SCEL,KMIA,AAL,2017-01-04 23:33:00,226,SCEL,KMIA,AAL,4,1,2017,Miercoles,I,American Airlines,Santiago,Miami,3.0,0
4,2017-01-05 23:30:00,226,SCEL,KMIA,AAL,2017-01-05 23:28:00,226,SCEL,KMIA,AAL,5,1,2017,Jueves,I,American Airlines,Santiago,Miami,-2.0,0


In [30]:
# temporada_alta : 1 si Fecha-I está entre 15-Dic y 3-Mar, o 15-Jul y 31-Jul, o 11-Sep y 30-Sep, 0 si no.

def temporada(x):
    x = pd.to_datetime(x, format = '%d-%m').strftime('%d-%m')
    if (x>='15-12') and (x <= '03-03'):
        return 1
    if (x>='15-07') and (x <= '31-07'):
        return 1
    if (x>='11-09') and (x <= '30-09'):
        return 1
    else:
        return 0
    

In [31]:
df['temporada_alta'] = df['Fecha-I'].apply(temporada)

In [17]:
 
#df['Fecha-I']= pd.to_datetime(df['Fecha-I'], format = '%d-%m').dt.strftime('%d-%m')

In [32]:
df.head(60)

,Fecha-I,Vlo-I,Ori-I,Des-I,Emp-I,Fecha-O,Vlo-O,Ori-O,Des-O,Emp-O,...,MES,AÑO,DIANOM,TIPOVUELO,OPERA,SIGLAORI,SIGLADES,dif_min,atraso_15,temporada_alta
0,2017-01-01 23:30:00,226,SCEL,KMIA,AAL,2017-01-01 23:33:00,226,SCEL,KMIA,AAL,...,1,2017,Domingo,I,American Airlines,Santiago,Miami,3.0,0,0
1,2017-01-02 23:30:00,226,SCEL,KMIA,AAL,2017-01-02 23:39:00,226,SCEL,KMIA,AAL,...,1,2017,Lunes,I,American Airlines,Santiago,Miami,9.0,0,0
2,2017-01-03 23:30:00,226,SCEL,KMIA,AAL,2017-01-03 23:39:00,226,SCEL,KMIA,AAL,...,1,2017,Martes,I,American Airlines,Santiago,Miami,9.0,0,0
3,2017-01-04 23:30:00,226,SCEL,KMIA,AAL,2017-01-04 23:33:00,226,SCEL,KMIA,AAL,...,1,2017,Miercoles,I,American Airlines,Santiago,Miami,3.0,0,0
4,2017-01-05 23:30:00,226,SCEL,KMIA,AAL,2017-01-05 23:28:00,226,SCEL,KMIA,AAL,...,1,2017,Jueves,I,American Airlines,Santiago,Miami,-2.0,0,0
5,2017-01-06 23:30:00,226,SCEL,KMIA,AAL,2017-01-07 00:04:00,226,SCEL,KMIA,AAL,...,1,2017,Sabado,I,American Airlines,Santiago,Miami,34.0,1,0
6,2017-01-07 23:30:00,226,SCEL,KMIA,AAL,2017-01-07 23:25:00,226,SCEL,KMIA,AAL,...,1,2017,Sabado,I,American Airlines,Santiago,Miami,-5.0,0,0
7,2017-01-08 23:30:00,226,SCEL,KMIA,AAL,2017-01-08 23:25:00,226,SCEL,KMIA,AAL,...,1,2017,Domingo,I,American Airlines,Santiago,Miami,-5.0,0,0
8,2017-01-09 23:30:00,226,SCEL,KMIA,AAL,2017-01-10 00:10:00,226,SCEL,KMIA,AAL,...,1,2017,Martes,I,American Airlines,Santiago,Miami,40.0,1,0
9,2017-01-11 23:30:00,226,SCEL,KMIA,AAL,2017-01-12 00:39:00,226,SCEL,KMIA,AAL,...,1,2017,Jueves,I,American Airlines,Santiago,Miami,69.0,1,0


In [35]:
#periodo_dia : mañana (entre 5:00 y 11:59), tarde (entre 12:00 y 18:59) y noche (entre 19:00 y 4:59), en base a Fecha-I
def periodos(x):
    x = pd.to_datetime(x, format = '%H:%M').strftime('%H:%M')
    if (x>='05:00') and (x <= '11:59'):
        return 'mañana'
    if (x>='12:00') and (x <= '18:59'):
        return 'tarde'  
    else:
        return 'noche' 


In [36]:
df['periodo_dia'] = df['Fecha-I'].apply(periodos)

In [39]:
df.tail(60)

,Fecha-I,Vlo-I,Ori-I,Des-I,Emp-I,Fecha-O,Vlo-O,Ori-O,Des-O,Emp-O,...,AÑO,DIANOM,TIPOVUELO,OPERA,SIGLAORI,SIGLADES,dif_min,atraso_15,temporada_alta,periodo_dia
68146,2017-12-22 15:06:00,253,SCEL,SCTE,JAT,2017-12-22 15:36:00,253.0,SCEL,SCTE,JAT,...,2017,Viernes,N,JetSmart SPA,Santiago,Puerto Montt,30.0,1,1,tarde
68147,2017-12-23 07:50:00,253,SCEL,SCTE,JAT,2017-12-23 07:54:00,253.0,SCEL,SCTE,JAT,...,2017,Sabado,N,JetSmart SPA,Santiago,Puerto Montt,4.0,0,1,mañana
68148,2017-12-24 16:04:00,253,SCEL,SCTE,JAT,2017-12-24 16:03:00,253.0,SCEL,SCTE,JAT,...,2017,Domingo,N,JetSmart SPA,Santiago,Puerto Montt,-1.0,0,1,tarde
68149,2017-12-28 14:06:00,253,SCEL,SCTE,JAT,2017-12-28 14:05:00,253.0,SCEL,SCTE,JAT,...,2017,Jueves,N,JetSmart SPA,Santiago,Puerto Montt,-1.0,0,1,tarde
68150,2017-12-29 15:06:00,253,SCEL,SCTE,JAT,2017-12-29 15:07:00,253.0,SCEL,SCTE,JAT,...,2017,Viernes,N,JetSmart SPA,Santiago,Puerto Montt,1.0,0,1,tarde
68151,2017-12-30 07:50:00,253,SCEL,SCTE,JAT,2017-12-30 07:45:00,253.0,SCEL,SCTE,JAT,...,2017,Sabado,N,JetSmart SPA,Santiago,Puerto Montt,-5.0,0,1,mañana
68152,2017-12-31 16:04:00,253,SCEL,SCTE,JAT,2017-12-31 16:11:00,253.0,SCEL,SCTE,JAT,...,2017,Domingo,N,JetSmart SPA,Santiago,Puerto Montt,7.0,0,0,tarde
68153,2017-12-19 18:51:00,255,SCEL,SCTE,JAT,2017-12-19 19:14:00,255.0,SCEL,SCTE,JAT,...,2017,Martes,N,JetSmart SPA,Santiago,Puerto Montt,23.0,1,1,tarde
68154,2017-12-21 18:26:00,255,SCEL,SCTE,JAT,2017-12-21 18:32:00,255.0,SCEL,SCTE,JAT,...,2017,Jueves,N,JetSmart SPA,Santiago,Puerto Montt,6.0,0,1,tarde
68155,2017-12-25 05:30:00,255,SCEL,SCTE,JAT,2017-12-25 05:42:00,255.0,SCEL,SCTE,JAT,...,2017,Lunes,N,JetSmart SPA,Santiago,Puerto Montt,12.0,0,1,mañana
